In [ ]:
# 1000 topics (age)
# Train RMSE: 9.095497197568372
# Test RMSE: 10.460199129743534
# R2 Train: 0.3698023817559233
# R2 Test: 0.230573536354301

# 2000 topics (age)
# Train RMSE: 8.611894270144457
# Test RMSE: 10.43693839611697
# R2 Train: 0.4350353857658916
# R2 Test: 0.2339917359910033

# 1000 topics (isFemale)
# Train Error: 0.17829181494661916
# Test Error: 0.23755334281650076

# 2000 topics (isFemale)
# Train Error: 0.14270462633451952
# Test Error: 0.2304409672830725

In [6]:
import warnings
warnings.filterwarnings("ignore")

# twitter

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import copy
import numpy as np
import torch
import tqdm
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

# change this
dataset = 'twitter' # folder and dataset name

cols = ['age', 'politics']

lr = 0.1

# START

train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
df = pd.read_csv('data/' + dataset + '.csv') # loading in actual dataset

label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])

merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
merged.columns = ['message_id', 'user_id', 'message'] + cols

user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
all_user_ids = list(user_ids.keys())

labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
user_ids_avg_dict = {} # populated to be user_id : average distribution

all_ys = []
for key in user_ids.keys():
    all_ys.append(labels_data[labels_data['user_id'] == key][cols].iloc[0])

for outcome in cols:
    print(outcome)
    print()

    y = [i[outcome] for i in all_ys]
    
    # REGRESSION

    all_scores = {}

    for model in ['BERTopic']:
        print(model)
        
        train_error_list = []
        test_error_list = []

        r2s_train = []
        r2s_test = []

        for run in range(1, 3):
            print("Run: " + str(run))
            print()

            # topics = []
            # with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_2000.txt', 'r') as f:
            #     reader = csv.reader(f)
            #     for row in reader:
            #         topic_list = [item.strip() for item in row if item.strip()]
            #         topics.append(topic_list)

            X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

            # 80-20 split --> didn't use train-test-split function since its already shuffled
            X_train = X[:round(0.80 * len(X))]
            X_test = X[round(0.80 * len(X)):]

            y_train = y[:round(0.80 * len(X))]
            y_test = y[round(0.80 * len(X)):]

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            torch.manual_seed(42) # for reproducibility

            X_train_NN = torch.tensor(X_train, dtype = torch.float32)
            y_train_NN = torch.tensor(y_train, dtype = torch.float32).reshape(-1, 1)
            X_test_NN = torch.tensor(X_test, dtype = torch.float32)
            y_test_NN = torch.tensor(y_test, dtype = torch.float32).reshape(-1, 1)

            model_NN = nn.Sequential(
                nn.Linear(X_train_NN.shape[1], 1),
            )

            loss_fn = nn.MSELoss()
            optimizer = optim.Adam(model_NN.parameters(), lr = lr)

            n_epochs = 1000
            batch_size = 32
            batch_start = torch.arange(0, len(X_train), batch_size)

            best_mse = np.inf
            best_weights = None
            history = []

            for epoch in range(n_epochs):
                model_NN.train()
                with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
                    bar.set_description(f"Epoch {epoch}")
                    for start in bar:

                        X_batch = X_train_NN[start : start+batch_size]
                        y_batch = y_train_NN[start : start+batch_size]

                        y_pred = model_NN(X_batch)
                        loss = loss_fn(y_pred, y_batch)

                        optimizer.zero_grad()
                        loss.backward()

                        optimizer.step()

                        bar.set_postfix(mse=float(loss))

                model_NN.eval()
                y_pred = model_NN(X_test_NN)
                mse = loss_fn(y_pred, y_test_NN)
                mse = float(mse)
                history.append(mse)
                if mse < best_mse:
                    best_mse = mse
                    best_weights = copy.deepcopy(model_NN.state_dict())

                if epoch % 50 == 0:
                    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

            model_NN.load_state_dict(best_weights)

            y_pred_train = model_NN(X_train_NN).detach().numpy()
            y_pred_test = model_NN(X_test_NN).detach().numpy()

            train_error = np.sqrt(mean_squared_error(y_pred_train, y_train))
            test_error = np.sqrt(mean_squared_error(y_pred_test, y_test))

            train_error_list.append(train_error)
            test_error_list.append(test_error)

            print(f'Train RMSE: {train_error}')
            print(f'Test RMSE: {test_error}')

            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)

            r2s_train.append(r2_train)
            r2s_test.append(r2_test)
            
            print(f'R2 Train: {r2_train}')
            print(f'R2 Test: {r2_test}')

            all_scores[model] = {
                'Train RMSE': train_error_list,
                'Test RMSE': test_error_list,
                'R2 Train': r2s_train,
                'R2 Test': r2s_test,

                'y_train': y_train,
                'y_train_pred': y_pred_train,
                'y_test': y_test,
                'y_test_pred': y_pred_test
            }
        
        print()

    print()

    for m in all_scores.keys():
        print(f'{m} Average Train RMSE: {np.mean(all_scores[m]["Train RMSE"])}')
        print(f'{m} Average Test RMSE: {np.mean(all_scores[m]["Test RMSE"])}')
        print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
        print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
        print()

    # with open('all_results/' + dataset + '_all_scores_' + outcome + '_NN.pkl', 'wb') as f:
    #     pickle.dump(all_scores, f)

age

BERTopic
Run: 2

Epoch 1, Loss: 524.6688842773438
Epoch 51, Loss: 88.45242309570312
Epoch 101, Loss: 72.90101623535156
Epoch 151, Loss: 65.9138412475586
Epoch 201, Loss: 61.97526168823242
Epoch 251, Loss: 59.255821228027344
Epoch 301, Loss: 57.12215042114258
Epoch 351, Loss: 55.32066345214844
Epoch 401, Loss: 53.732112884521484
Epoch 451, Loss: 52.293758392333984
Epoch 501, Loss: 50.96934127807617
Epoch 551, Loss: 49.73602294921875
Epoch 601, Loss: 48.578590393066406
Epoch 651, Loss: 47.48653793334961
Epoch 701, Loss: 46.452674865722656
Epoch 751, Loss: 45.471519470214844
Epoch 801, Loss: 44.53873062133789
Epoch 851, Loss: 43.65091323852539
Epoch 901, Loss: 42.80537796020508
Epoch 951, Loss: 41.999977111816406
Train RMSE: 8.611894270144457
Test RMSE: 10.43693839611697
R2 Train: 0.4350353857658916
R2 Test: 0.2339917359910033


BERTopic Average Train RMSE: 8.611894270144457
BERTopic Average Test RMSE: 10.43693839611697
BERTopic Average R2 Train: 0.4350353857658916
BERTopic Average R

In [8]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

# change this
dataset = 'twitter' # folder and dataset name
cols = ['gender'] # outcome columns
outcome = 'gender'
buckets = 4

all_scores = {}

train = pickle.load(open(dataset + '/BERTopic/run_1/train.pkl', 'rb'))
test = pickle.load(open(dataset + '/BERTopic/run_1/test.pkl', 'rb'))

temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
df = pd.read_csv('data/' + dataset + '.csv') # loading in actual dataset

label_encoder = LabelEncoder()
df['gender'] = label_encoder.fit_transform(df['gender'])

merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
merged.columns = ['message_id', 'user_id', 'message'] + cols

user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
all_user_ids = list(user_ids.keys())

labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
user_ids_avg_dict = {} # populated to be user_id : average distribution
y = [] # the outcome we care about

for key in user_ids.keys():
    y.append(labels_data[labels_data['user_id'] == key][outcome].iloc[0])

num_epochs = 500

all_scores = {}

for model in ['BERTopic']:
    print(model)

    train_error_list = []
    test_error_list = []

    r2s_train = []
    r2s_test = []

    if model == 'GensimLDA':
        lr = 0.5
    elif model == 'BERTopic':
        lr = 0.05
    elif model == 'NMF':
        lr = 0.05
    elif model == 'Mallet_LDA':
        lr = 0.1
    else:
        lr = 0.1

    for run in range(1, 3):
        print("Run: " + str(run))
        print()

        # topics = []
        # with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_2000.txt', 'r') as f:
        #     reader = csv.reader(f)
        #     for row in reader:
        #         topic_list = [item.strip() for item in row if item.strip()]
        #         topics.append(topic_list)

        X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

        # # 80-20 split --> didn't use train-test-split function since its already shuffled
        X_train = X[:round(0.80 * len(X))]
        X_test = X[round(0.80 * len(X)):]

        y_train = y[:round(0.80 * len(X))]
        y_test = y[round(0.80 * len(X)):]

        X_train = np.array(X_train)
        X_test = np.array(X_test)
        y_train = np.array(y_train)
        y_test = np.array(y_test)

        train_users = all_user_ids[:round(0.80 * len(X))]
        test_users = all_user_ids[round(0.80 * len(X)):]

        # Convert arrays to torch tensors
        X_train_tensor = torch.tensor(np.array(X_train).astype(np.float32))
        y_train_tensor = torch.tensor(np.array(y_train).astype(np.longlong))  # Use long for classification
        X_test_tensor = torch.tensor(np.array(X_test).astype(np.float32))
        y_test_tensor = torch.tensor(np.array(y_test).astype(np.longlong))

        # Create datasets and dataloaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)
        test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

        # Define the model
        class LogisticRegressionModel(nn.Module):
            def __init__(self, input_size, num_classes):
                super(LogisticRegressionModel, self).__init__()
                self.layer1 = nn.Linear(input_size, num_classes)

            def forward(self, x):
                return self.layer1(x)

        input_size = X_train.shape[1]
        num_classes = len(np.unique(y_train))  # Assuming y_train contains all classes
        logit_model = LogisticRegressionModel(input_size, num_classes)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()  # This includes softmax
        optimizer = optim.Adam(logit_model.parameters(), lr=lr)

        loss_values = []
        early_stopping_triggered = False

        # Train the model
        for epoch in range(num_epochs):
            epoch_loss = 0
            for inputs, targets in train_loader:
                # Forward pass
                outputs = logit_model(inputs)
                loss = criterion(outputs, targets)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item() * inputs.size(0) 
            
            epoch_loss /= len(train_loader.dataset)
            loss_values.append(epoch_loss)

            if epoch >= 50:
                # Calculate the percentage change in loss
                loss_change = (loss_values[epoch - 50] - loss_values[epoch]) / loss_values[epoch - 50]
                
                # If change in loss is less than 1%, stop training
                if abs(loss_change) < 0.005:
                    print(f'Early stopping at epoch {epoch+1} due to minimal loss improvement.')
                    early_stopping_triggered = True
                    break

            if (epoch+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Predict on the test set
        logit_model.eval()  # Set the model to evaluation mode

        with torch.no_grad():
            y_pred_train = logit_model(X_train_tensor)
            _, predicted_train = torch.max(y_pred_train.data, 1)

            y_pred_test = logit_model(X_test_tensor)
            _, predicted_test = torch.max(y_pred_test.data, 1)


        train_error = 1 - accuracy_score(predicted_train, y_train)
        test_error = 1 - accuracy_score(predicted_test, y_test)

        train_error_list.append(train_error)
        test_error_list.append(test_error)
        
        print(f'Train Error: {train_error}')
        print(f'Test Error: {test_error}')

        print()

        r2_train = r2_score(y_train, predicted_train)
        r2_test = r2_score(y_test, predicted_test)

        r2s_train.append(r2_train)
        r2s_test.append(r2_test)
        
        print(f'R2 Train: {r2_train}')
        print(f'R2 Test: {r2_test}')

        all_scores[model] = {
            'Train Error': train_error_list,
            'Test Error': test_error_list,
            'R2 Train': r2s_train,
            'R2 Test': r2s_test,

            'y_train': y_train,
            'y_train_pred': predicted_train.numpy(),
            'y_test': y_test,
            'y_test_pred': predicted_test.numpy()
        }

    print()

print()

for m in all_scores.keys():
    print(f'{m} Average Train Error: {np.mean(all_scores[m]["Train Error"])}')
    print(f'{m} Average Test Error: {np.mean(all_scores[m]["Test Error"])}')
    print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
    print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
    print()

# with open('all_results/' + dataset + '_all_scores_' + outcome + '.pkl', 'wb') as f:
#      pickle.dump(all_scores, f)

BERTopic
Run: 2

Epoch [10/500], Loss: 0.6279
Epoch [20/500], Loss: 0.5996
Epoch [30/500], Loss: 0.5756
Epoch [40/500], Loss: 0.5548
Epoch [50/500], Loss: 0.5366
Epoch [60/500], Loss: 0.5206
Epoch [70/500], Loss: 0.5064
Epoch [80/500], Loss: 0.4937
Epoch [90/500], Loss: 0.4822
Epoch [100/500], Loss: 0.4718
Epoch [110/500], Loss: 0.4624
Epoch [120/500], Loss: 0.4538
Epoch [130/500], Loss: 0.4459
Epoch [140/500], Loss: 0.4386
Epoch [150/500], Loss: 0.4320
Epoch [160/500], Loss: 0.4258
Epoch [170/500], Loss: 0.4201
Epoch [180/500], Loss: 0.4147
Epoch [190/500], Loss: 0.4098
Epoch [200/500], Loss: 0.4051
Epoch [210/500], Loss: 0.4008
Epoch [220/500], Loss: 0.3967
Epoch [230/500], Loss: 0.3929
Epoch [240/500], Loss: 0.3892
Epoch [250/500], Loss: 0.3858
Epoch [260/500], Loss: 0.3826
Epoch [270/500], Loss: 0.3795
Epoch [280/500], Loss: 0.3765
Epoch [290/500], Loss: 0.3737
Epoch [300/500], Loss: 0.3711
Epoch [310/500], Loss: 0.3685
Epoch [320/500], Loss: 0.3660
Epoch [330/500], Loss: 0.3637
Ep